# Exercise 1
Obtain the general formulation for the Google Adwords problem (described within slides 23-26 of the Linear Programming topic).

**Data:**

$a_i = $ maximum budget for company

$b_j = $ maximum of expected request for query

$c_{ij} = $ price per add/click

**Variables:**

$x_{ij} = $ Number of adds assigned per query and company

$$\max_{x_{ij}} \quad \sum_{i=1}^n \sum_{j=1}^m c_{ij} x_{ij}$$

$$s.t. \quad \sum_{i=1}^n x_{ij} \leq b_j \quad \forall j$$
$$\sum_{j=1}^m c_{ij} x_{ij} \leq a_i \quad \forall i$$
$$x_{ij} \geq 0 \quad ij$$

# Exercise 2
Obtain its equivalent standard form.

$$ \max_x \quad z = c^{\top}x $$
$$s.t. \quad Ax = b $$
$$x \geq =0 $$

where $x \in {\rm I\!R}^n, c \in {\rm I\!R}^n, A \in {\rm I\!R}^{m\times n}, and b \in {\rm I\!R}^m$


# Exercise 3
Implement the model derived in 2) in Pyomo and solve it for the case of 10 companies and 10 queries (make up the data to be reasonable). Compute the sensitivities associated to each constraint. Report the results.

In [5]:
%%writefile 1st_assignement3.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

# Future number must be NonNegative.
model.n = Param(within=NonNegativeIntegers) #number of Companies 
model.m = Param(within=NonNegativeIntegers) #number of Queries


model.I = RangeSet(1,model.n)
model.J = RangeSet(1,model.m)


model.r=Param(model.I, model.J) #Google Revenues
model.q=Param(model.J) #Estimated number of requests
model.b=Param(model.I) #Budget


model.x=Var(model.I,model.J, domain=NonNegativeReals)

#Objective
def obj_expression(model):
    return summation(model.r,model.x)
model.OBJ= Objective(rule=obj_expression, sense= maximize)

#Constraints:
def q_constraint(model,j):
    return sum(model.x[i,j] for i in model.I)<=model.q[j]

model.que_cons=Constraint(model.J, rule=q_constraint)


def b_constraint(model,i):
    return sum(model.r[i,j]*model.x[i,j] for j in model.J)<=model.b[i]

model.comp_cons=Constraint(model.I, rule=b_constraint)


Overwriting 1st_assignement3.py


In [20]:
%%writefile 1st_assignement3.dat


param n :=3; #Companies
param m :=3; #Queries
param q :=
    1 150
    2 90
    3 80
    ;
param b:=
    1 200
    2 151
    3 180
    ;
param r: 1 2 3 :=
1 1 0.75 5
2 0.5 0.5 2
3 0.5 3 1
;

Overwriting 1st_assignement3.dat


In [23]:
!pyomo solve 1st_assignement3.py 1st_assignement3.dat --solver=glpk --summary --solver-suffix=dual

usage: pyomo solve [options] <model_or_config_file> [<data_files>]
pyomo solve: error: unrecognized arguments: --solver-suffixes=dual


In [22]:
!type results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 531.0
  Upper bound: 531.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 10
  Number of nonzeros: 19
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0259997844696
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------------

In [ ]:
%%writefile 1st_assignement3.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

# Future number must be NonNegative.
model.n = Param(within=NonNegativeIntegers) #number of Companies 
model.m = Param(within=NonNegativeIntegers) #number of Queries


model.I = RangeSet(1,model.n)
model.J = RangeSet(1,model.m)


model.r=Param(model.I, model.J) #Google Revenues
model.q=Param(model.J) #Estimated number of requests
model.b=Param(model.I) #Budget


model.x=Var(model.I,model.J, domain=NonNegativeReals)

#Objective
def obj_expression(model):
    return summation(model.r,model.x)
model.OBJ= Objective(rule=obj_expression, sense= maximize)


#Constraints:
def q_constraint(model,j):
    return sum(model.x[i,j] for i in model.I)<=model.q[j]

model.que_cons=Constraint(model.J, rule=q_constraint)


def b_constraint(model,i):
    return sum(model.r[i,j]*model.x[i,j] for j in model.J)<=model.b[i]

model.comp_cons=Constraint(model.I, rule=b_constraint)

# Exercise 4
Given a linear programming problem in standard form:
$$\min_{x} \quad z_P = c^{\top} x $$ 
$$s.t. \quad Ax = b $$
$$x \geq 0 $$
we can define its dual problem as 
$$\max_{y} \quad z_D = b^{\top} y $$
$$s.t. \quad A^{\top}y\geq c $$
where $y$ is called teh dual variable vector.

Considering this, formulate the dual problem associated to the model derived in 2) (check slide 60 of the Linear Programming topic for an example of this transformation).

# Exercise 5
Implement the dual model derived in 4) in Pyomo and solve it for the same data in 3). Report the results.

# Exercise 6
The Strong Duality Theorem states that:

If $x^*$ is the optimal solution of the primal minimization LP problem, and $y^*$ is the dual optimal solution of the corresponding dual maximization LP problem, then

$$z^*_D = b^{\top} y^* = c^{\top} x^* = z^*_P $$

Comparing the solutions in 3) and 5), check if the Strong Duality Theorem holds. What is the relationship between the sensitivities computed in 3) and the optimal value of the dual variables obtained in 5)?

# Exercise 7
Imagine now that google is able to display simultaneously n company ads for each requested query (instead of only one). Moreover, assume that the specific order in which these ads are displayed is important. Indicate how the model in 1) would need to be modified to account for these facts. Also, indicate if any additional problem data will be necessary for this new setting.

# IMPORTANT:

Upload the formulations 1), 2), 4) and 7) as well as the answers to the different questions in a pdf file (generated with MSWord, latex or similar) and the codes for 3) and 5) as separated .py and .dat files.

This assignment can be done individually or in groups (up to 4 components). Groups must be the same for all the assignments. Files need to be uploaded by only one member of the group. Do not forget to indicate in the pdf file all the group’s components.